https://github.com/huggingface/notebooks/blob/main/sagemaker/28_train_llms_with_qlora/sagemaker-notebook.ipynb

In [19]:
#.env

In [1]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])
# dataset size: 15011



Found cached dataset json (C:/Users/Pericles/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


dataset size: 15011
{'instruction': 'What is appealing about Charleston in South Carolina?', 'context': '', 'response': 'Charleston has a lot of historical sites that you can visit, which is great for educational purposes. The food is very diverse and provides many options for the pickiest of eaters. The low country is a beautiful scenery that sets the scene for cocktails, sunset catching and listening to the sounds of the creatures. There is a lot to see and learn from Charleston because of its southern charm and the people that have built its culture into a blossoming city.', 'category': 'general_qa'}


In [2]:
def format_dolly(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Answer\n{sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt

In [3]:
from random import randrange

print(format_dolly(dataset[randrange(len(dataset))]))

### Instruction
What are the main characteristics of a Haiku poem?

### Context
Haiku (俳句, listen (help·info)) is a type of short form poetry originally from Japan. Traditional Japanese haiku consist of three phrases composed of 17 phonetic units (called on in Japanese, which are similar to syllables) in a 5, 7, 5 pattern; that include a kireji, or "cutting word"; and a kigo, or seasonal reference. Similar poems that do not adhere to these rules are generally classified as senryū.

Haiku originated as an opening part of a larger Japanese poem called renga. These haiku written as an opening stanza were known as hokku and over time they began to be written as stand-alone poems. Haiku was given its current name by the Japanese writer Masaoka Shiki at the end of the 19th century.

Originally from Japan, haiku today are written by authors worldwide. Haiku in English and haiku in other languages have different styles and traditions while still incorporating aspects of the traditional haiku f

In [4]:
from transformers import AutoTokenizer

model_id = "TinyPixel/Llama-2-7B-bf16-sharded" # sharded weights
#model_id = "Xilabs/Llama-2-7b-Sharded" # sharded weights

tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token

z:\Projetos\LLM\.env\Lib\site-packages\transformers\models\auto\tokenization_auto.py:631: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [5]:
from random import randint
from itertools import chain
from functools import partial


# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample


# apply prompt template per sample
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# print random sample
print(dataset[randint(0, len(dataset))]["text"])

# empty list to save remainder from batches to use in next batch
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # define global remainder variable to save remainder from batches to use in next batch
    global remainder
    # Concatenate all texts and add remainder from previous batch
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # get total number of tokens for batch
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # get max number of chunks for batch
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # Split by chunks of max_len.
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # add remainder to global variable for next batch
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result


# tokenize and chunk dataset
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# Print total number of samples
print(f"Total number of samples: {len(lm_dataset)}")

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

### Instruction
Tell me whether these are European countries or Asian countries: Spain, France, Norway, the Czech Republic, Denmark, Portugal, Brazil, South Africa, Canada, China, Japan, Malaysia, Singapore, Sri Lanka, The Philippines

### Answer
Not all of them are European countries or Asian countries! Brazil, South Africa, Canada are not part of Europe or Asia. 

These are the European countries: Spain, France, Norway, the Czech Republic, Denmark, Portugal

And these are Asian countries: China, Japan, Malaysia, Singapore, Sri Lanka, The Philippines</s>


Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

Total number of samples: 1591


In [7]:
lm_dataset.save_to_disk("modelo_treinado")


Saving the dataset (0/1 shards):   0%|          | 0/1591 [00:00<?, ? examples/s]